In [16]:
%load_ext autoreload
%autoreload 2

import numpy as np
import nibabel as nb

import sys
sys.path.append("..")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
from prfpy.stimulus import PRFStimulus2D
from prfpy.grid import Iso2DGaussianGridder, Norm_Iso2DGaussianGridder
from prfpy.fit import Iso2DGaussianFitter, Norm_Iso2DGaussianFitter

from utils.utils import *

In [ ]:
dm_1 = create_dm_from_screenshots(screenshot_path='')
dm_2 = create_dm_from_screenshots(screenshot_path='')
dm_3 = create_dm_from_screenshots(screenshot_path='')
dm_4 = create_dm_from_screenshots(screenshot_path='')
dm_5 = create_dm_from_screenshots(screenshot_path='')

dm_full = np.concatenate((dm1,dm2,dm3,dm4,dm5) axis=-1)

prf_stim = PRFStimulus2D(screen_size_cm=70, 
                         screen_distance_cm=210, 
                         design_matrix=dm_full,
                         TR=1.5)

grid_nr = 20
max_ecc_size = 16
sizes, eccs, polars = max_ecc_size * np.linspace(0.25,1,grid_nr)**2, \
                    max_ecc_size * np.linspace(0.1,1,grid_nr)**2, \
                        np.linspace(0, 2*np.pi, grid_nr)

gg = Iso2DGaussianGridder(stimulus=prf_stim)

In [ ]:
##TODO: SETUP ANALYSIS AS IN NOTES

## TEMPORARY
mask = nb.load('/Users/marcoaqil/PRFMapping/PRFMapping-Deriv-nomprage-durab7.5-hires/fmriprep/sub-001/ses-1/func/sub-001_ses-1_task-1R_run-1_space-T1w_desc-aparcaseg_dseg.nii.gz')
mask_cortex = mask.get_data().astype(bool)
timecoursefile = nb.load('/Users/marcoaqil/PRFMapping/PRFMapping-Deriv-nomprage-durab7.5-hires/fmriprep/sub-001/ses-1/func/sub-001_ses-1_task-1R_run-1_space-T1w_desc-preproc_bold.nii.gz')
timecourse = timecoursefile.get_data()
timecourse_cortex = timecourse[mask_cortex]
#exclude timecourses with zero variance
timecourse_cortex_nonzerovar = timecourse_cortex[np.where(np.var(timecourse_cortex, axis=-1)>0)]

In [ ]:
%%time
gf = Iso2DGaussianFitter(data=timecourse_cortex_nonzerovar, gridder=gg, n_jobs=10)

gf.grid_fit(ecc_grid=eccs,
                 polar_grid=polars,
                 size_grid=sizes)

In [ ]:
#refine Gaussian fits
%%time
gf.n_jobs=10
gf.iterative_fit(rsq_threshold=0.0, verbose=True)

In [ ]:
%%time
#now refit normalization model, starting from results of iterated Gaussian fitting
gg_norm = Norm_Iso2DGaussianGridder(stimulus=prf_stim)
gf_norm = Norm_Iso2DGaussianFitter(data=timecourse_cortex_nonzerovar,
                                   gridder=gg_norm,
                                   n_jobs=10)

#have to add a column since in current code syntax
#gridsearch_params always contains the CSS exponent parameter, even if it is not fit.
#whereas iterative_search_params does not contain it if it is not fit)

starting_params = np.insert(gf.iterative_search_params, -1, 1.0, axis=-1)

gf_norm.iterative_fit(rsq_threshold=0.05, gridsearch_params=starting_params, verbose=True)

current_result=np.copy(gf_norm.iterative_search_params)

In [ ]:
%%time
#if needed, the iterative fit can be run again 
#(this makes sense only if changing some values in iterative_search minimization for increased precision)
new_starting_params = np.insert(current_result, -1, 1.0, axis=-1)
gf_norm.iterative_fit(rsq_threshold=0.0, gridsearch_params=new_starting_params, verbose=True)
current_result=np.copy(gf_norm.iterative_search_params)

In [ ]:
np.mean(gf_norm.iterative_search_params[gf_norm.rsq_mask,-1])

In [ ]:
np.mean(gf.iterative_search_params[gf_norm.rsq_mask,-1])

In [ ]:
attempt = np.zeros((78,102,80,10))
ha = attempt.reshape((-1,10))
ha[np.ravel(np.var(timecourse, axis=-1)>0) & np.ravel(mask_cortex)]=gf_norm.iterative_search_params

haha = ha.reshape((78,102,80,10))

for i in range(10):
    nb.Nifti1Image(haha[:,:,:,i], timecoursefile.affine).to_filename('norm{}.nii.gz'.format(i))
